In [ ]:
import os

import numpy as np
import pandas as pd
import pyreadr

import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from gglasso.helper.utils import sparsity, zero_replacement, normalize, log_transform
from gglasso.problem import glasso_problem

### Import preprocessed soil data

In [ ]:
metadata = pd.read_table('../../data/soil/88soils_modified_metadata.txt', index_col=0)
metadata.head(2)

In [ ]:
soil = pd.read_csv('../../data/soil/soil_id_116.csv', sep=',', index_col = 0).T
soil.head()

In [ ]:
ph = metadata["ph"]
ph = ph.reindex(soil.columns)
ph.head()

In [ ]:
depth = soil.sum(axis=0)

In [ ]:
#check if any ids are missing
assert not ph.isnull().values.any()

# assert that samples of ph and soil are identical
assert set(soil.columns) == set(ph.index)

### CLR-transformation of X

##### Dataframe `soil` need to be of shape (p,N) for normalizing to simplex + clr transform

In [ ]:
soil.shape

In [ ]:
X = normalize(soil)
X.sum(axis=0)

In [ ]:
X = log_transform(X)
(p,N) = X.shape
(p,N)

### Calculate covariance and scale to correlations

In [ ]:
S0 = np.cov(X.values, bias = True)
# scale covariances to correlations
scale = np.tile(np.sqrt(np.diag(S0)),(S0.shape[0],1))
scale = scale.T * scale

S = S0 / scale
np.diag(S)

### GGLasso problem and model selection

In [ ]:
P = glasso_problem(S, N, latent = True, do_scaling = False)
print(P)

#lambda1_range = [0.14447343]
#mu1_range = [2.36]

lambda1_range = np.logspace(0.5,-1.5,8)
mu1_range = np.logspace(1.5,-0.2,6)


modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range}

P.model_selection(modelselect_params = modelselect_params, method = 'eBIC', gamma = 0.25)

# regularization parameters are set to the best ones found during model selection
print(P.reg_params)

### Plot results from model selection

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(18,12))
sns.heatmap(P.modelselect_stats["RANK"], annot = True, square = True, cbar = False, \
            yticklabels = np.round(lambda1_range,2), xticklabels = np.round(mu1_range,2), ax = axs[0])
axs[0].set_title("Rank of L")
sns.heatmap(np.round(P.modelselect_stats["SP"],2), annot = True, square = True, cbar = False, \
            yticklabels = np.round(lambda1_range,2), xticklabels = np.round(mu1_range,2), ax = axs[1])
axs[1].set_title("Sparsity of Theta")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
def single_surface_plot(L1, MU1, C, ax, name = 'eBIC'):
    
    X = np.log10(L1)
    Y = np.log10(MU1)
    Z = np.log(C)
    ax.plot_surface(X, Y, Z , cmap = plt.cm.ocean, linewidth=0, antialiased=True)
    
    ax.set_xlabel(r'$\lambda_1$', fontsize = 14)
    ax.set_ylabel(r'$\mu1$', fontsize = 14)
    ax.set_zlabel(name, fontsize = 14)
    ax.view_init(elev = 18, azim = 51)
    
    plt.xticks(fontsize = 8)
    plt.yticks(fontsize = 8)
    ax.zaxis.set_tick_params(labelsize=8)
    
    ax.tick_params(axis='both', which='major', pad=.5)
    
    for label in ax.xaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    for label in ax.yaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    for label in ax.zaxis.get_ticklabels()[::2]:
        label.set_visible(False)
    
    return

fig = plt.figure(figsize = (20,10))  
fig.suptitle("eBIC surface for different gamma value")

C = P.modelselect_stats["BIC"]
gammas = np.sort(list(C.keys()))
        
for j in np.arange(len(gammas)):
    ax = fig.add_subplot(2, 3, j+1, projection='3d')
    single_surface_plot(P.modelselect_stats["LAMBDA"], P.modelselect_stats["MU"], C[gammas[j]], ax)
    if gammas is not None:
        ax.set_title(rf"$\gamma = $ {gammas[j]}")
    

The solution of Graphical Lasso with latent variables has the form $\Theta-L$ where $\Theta$ is sparse and $L$ has low rank.

In [ ]:
gg_lowrank = P.solution.lowrank_

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,8))
sns.heatmap(P.solution.precision_, ax = axs[0], cmap = "coolwarm", vmin = -0.5, vmax = 0.5, cbar = False, square = True)
axs[0].set_title("Heatmap of Theta")
sns.heatmap(gg_lowrank, ax = axs[1], cmap = "coolwarm", vmin = -0.05, vmax = 0.05, cbar = False, square = True)
axs[1].set_title("Heatmap of L")


In [ ]:
gg_rank = np.linalg.matrix_rank(gg_lowrank)
print('Rank of low-rank component: {0}'.format(gg_rank))
sig, V = np.linalg.eigh(gg_lowrank)

### Robust PCA in GGLasso

We use the low rank component of the Graphical Lasso solution in order to do a robust PCA. For this, we use the eigendecomposition

$$L = V \Sigma V^T$$

where the columns of $V$ are the orthonormal eigenvecors and $\Sigma$ is diagonal containing the eigenvalues.
Denote the columns of $V$ corresponding only to positive eigenvalues with $\tilde{V} \in \mathbb{R}^{p\times r}$ and $\tilde{\Sigma} \in \mathbb{R}^{r\times r}$ accordingly, where $r=\mathrm{rank}(L)$. Then we have 

$$L = \tilde{V} \tilde{\Sigma} \tilde{V}^T.$$

Now we project the data matrix $X\in \mathbb{R}^{p\times N}$ onto the eigenspaces of $L^{-1}$ - which are the same as of $L$ - by computing

$$U := X^T \tilde{V}\tilde{\Sigma}$$

We plot the columns of $U$ vs. the vector of pH values.

In [ ]:
def robust_PCA(X, L, inverse=True):
    sig, V = np.linalg.eigh(L)
    ind = np.argwhere(sig > 1e-9)

    if inverse:
        loadings = V[:,ind] @ np.diag(np.sqrt(1/sig[ind]))
    else:
        loadings = V[:,ind] @ np.diag(np.sqrt(sig[ind]))

    zu = X.values.T @ loadings
    
    return zu, loadings

### Plot GGLasso/pH correlation

In [ ]:
zu_gg, gg_loadings = robust_PCA(X, gg_lowrank, inverse=True)

In [ ]:
for i in range(gg_rank):
    plt.scatter(zu_gg[:,i], ph, c = depth, cmap = plt.cm.Blues, vmin = 0)
    cbar = plt.colorbar()
    cbar.set_label("Sampling depth")
    plt.xlabel(f"PCA component {i+1}")
    plt.ylabel("pH")
    plt.show()

In [ ]:
for i in range(0, gg_rank):
    print("Spearman correlation between pH and {0}th component: {1}, p-value: {2}".format(i+1, stats.spearmanr(ph, zu_gg[:,i])[0], 
                                                                              stats.spearmanr(ph, zu_gg[:,i])[1]))

## SpiecEasi results with lambda = 0.14447343, rank=6

In [ ]:
SE_lowrank = pyreadr.read_r('../../data/soil/SE_lowrank.rds')
SE_lowrank = np.array(SE_lowrank[None])
SE_lowrank.shape

In [ ]:
se_rank = np.linalg.matrix_rank(SE_lowrank)
print('Rank of low-rank component: {0}'.format(se_rank))

### Compare low rank SpiecEasi vs GGLasso

In [ ]:
np.allclose(SE_lowrank, gg_lowrank, atol=1e-1)

In [ ]:
fig, axs = plt.subplots(1,2, figsize = (20,8))
sns.heatmap(SE_lowrank, ax = axs[0], cmap = "coolwarm", vmin = -0.1, vmax = 0.1, cbar = False, square = True)
sns.heatmap(gg_lowrank, ax = axs[1], cmap = "coolwarm", vmin = -0.1, vmax = 0.1, cbar = False, square = True)

### Robust PCA in [SpiecEasi](https://github.com/zdk123/SpiecEasi/blob/ff528b23fafbd455efcca9dd356bef28951edf82/R/SparseLowRankICov.R)

### Plot SE/pH correlation

In [ ]:
zu_SE, se_loadings = robust_PCA(X, SE_lowrank, inverse=True)
zu_SE.shape

In [ ]:
for i in range(se_rank):
    plt.scatter(zu_SE[:,i], ph, c = depth, cmap = plt.cm.Blues, vmin = 0)
    cbar = plt.colorbar()
    cbar.set_label("Sampling depth")
    plt.xlabel(f"PCA component {i+1}")
    plt.ylabel("pH")
    plt.show()

In [ ]:
for i in range(se_rank):
    print("Spearman correlation between pH and {0}th component: {1}, p-value: {2}".format(i+1, stats.spearmanr(ph, zu_SE[:,i])[0], 
                                                                              stats.spearmanr(ph, zu_SE[:,i])[1]))